In [1]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------- ----------------------- 4.7/11.1 MB 28.6 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 36.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.9 MB ? eta -:--:--
   ----- ---------------------------------- 6.3/43.9 MB 42.8 MB/s eta 0:00:01
   ------------------ --------------------- 20.2/43.9 MB 49.1 MB/s eta 0:00:01
   -------------------------------- ------- 35.1/43.9 MB 55.8 MB/s eta 0:00:01
   ---------------------------------------  43.8/43.9 MB 54.6 MB/s eta 0:00:01
   ---------------------------------------- 43.9/43.9 MB 49.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv('Data_cleaned.csv')
# Séparation du DataFrame en train (80%) et test (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(df.columns)
# Affichage des dimensions des ensembles
print(f"Train: {train_df.shape}, Test: {test_df.shape}")
df = df.drop(columns=['Qualité du retour2'])
X = df.drop(columns=["Qualité du retour"])  # Variables explicatives
y = df["Qualité du retour"]               # Variable cible

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")

Index(['Date', 'Territoire', 'Sujet', 'Thème', 'Nbre d'article',
       'Qualité du retour', 'Média', 'Articles', 'Qualité du retour2',
       'Tonalité', 'sentiment', 'factuel'],
      dtype='object')
Train: (1100, 12), Test: (275, 12)
X_train: (1100, 10), X_test: (275, 10)
y_train: (1100,), y_test: (275,)


In [10]:
df.head()

,Date,Territoire,Sujet,Thème,Nbre d'article,Qualité du retour,Média,Articles,Tonalité,sentiment,factuel
0,01/02/2023,Pas-de-Calais,Une maison à l'abandon détruite dans un incendie,divers,2.0,factuel,La voix du Nord,Une maison à l'abandon détruite dans un incend...,factuel,neutre,True
1,01/02/2023,Pas-de-Calais,Trois familles relogés après un court-circuit,divers,1.0,factuel,La voix du Nord,Trois familles relogées après un court-circuit...,factuel,neutre,True
2,01/02/2023,Nord,"Suite à une panne, 179 foyers privés de courant",reseau,1.0,factuel negatif,La voix du Nord,"Suite à une panne, 179 foyers privés de couran...",factuel négatif,negatif,True
3,01/02/2023,Pas-de-Calais,"Plus de coupures d'éléctricté, mais des dégâts...",reseau,2.0,negatif,La voix du Nord,"Plus de coupures d'électricité, mais des dégât...",négatif,negatif,False
4,01/02/2023,Pas-de-Calais,Les décharges remplacés par deux centrales pho...,transition ecologique,1.0,factuel,La voix du Nord,les décharges remplacées par deux centrales\r...,factuel,neutre,True


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1️⃣ Chargement des données
df = pd.read_csv('Data_cleaned.csv')

# Suppression des colonnes inutiles
df = df.drop(columns=['Qualité du retour2'])  

# Définition des features (X) et de la target (y)
X = df["Texte"]  # Colonne contenant les textes
y = df["Qualité du retour"]  # Colonne cible (multi-classes)

# 2️⃣ Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3️⃣ Transformation TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='french', max_features=5000, ngram_range=(1,2))  
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 4️⃣ Entraînement d'un modèle plus puissant (Random Forest)
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train_tfidf, y_train)

# 5️⃣ Prédiction et évaluation
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}\n")
print("Classification Report:\n", classification_report(y_test, y_pred))

# 6️⃣ Fonction de prédiction pour un nouveau texte
def predict_text(text):
    text_tfidf = tfidf_vectorizer.transform([text])
    prediction = model.predict(text_tfidf)
    return prediction[0]

# Test sur un exemple
new_text = "Le service était excellent et rapide."
print(f"Prédiction pour le nouveau texte : {predict_text(new_text)}")


0               factuel
1               factuel
2       factuel negatif
3               negatif
4               factuel
             ...       
1370            factuel
1371     negatif nuance
1372            factuel
1373            factuel
1374            factuel
Name: Qualité du retour, Length: 1375, dtype: object
